# Estimation of piping failure probability (Bligh)

In this example, we will demonstrate the application of various reliability methods to estimate the probability of failure of a levee due to piping mechanism. The failure mechanism is characterized here using the simple model of Bligh.

The possible reliability methods are: form, numerical_integration, crude_monte_carlo, importance_sampling, adaptive_importance_sampling, subset_simulation.

The possible calculation settings are: variation_coefficient, relaxation_factor/loops, minimum/maximum_samples/iterations/directions/variance_loops, fraction_failed.

### Define model

First, we import the necessary package:

In [65]:
from ptk import *

The limit state function according to the piping model of Bligh is defined as follows:

$Z = \frac{m \cdot L}{c_{creep}} -\Delta H$

where: <br>

$\Delta H$ is the difference between the water level on the levee and the water level in the polder [m] <br>
$L$ is the seepage path length [m] <br>
$m$ represents model uncertainty [-] <br>
$c_{creep}$ is the creep factor [-]

The parameter $\Delta H$ represents the imposed load, while $L$ stands for the strength of the levee. The creep factor value for very fine-grained sand is $18$.

In [66]:
def bligh(m, L, c_creep, delta_H):
    return m * L / c_creep - delta_H

To perform a reliability analysis, we create a reliability project and specify the limit state function (model):

In [67]:
project = ReliabilityProject()
project.model = bligh

We assume the following distributions for the parameters $m$, $L$, $c_{creep}$ and $\Delta H$:

In [68]:
project.variables["m"].distribution = "log_normal"
project.variables["m"].mean = 1.76
project.variables['m'].deviation = 1.69

project.variables["L"].distribution = "normal"
project.variables["L"].mean = 50
project.variables["L"].deviation = 2.5

project.variables["c_creep"].distribution = "deterministic"
project.variables["c_creep"].mean = 18

project.variables["delta_H"].distribution = "gumbel"
project.variables["delta_H"].shift = 0.53
project.variables["delta_H"].scale = 0.406

### Perform reliability calculations with FORM

We start with the reliability method `FORM`. We choose the calculation settings: `relaxation_factor`, `variation_coefficient` and `maximum_iterations`. The reliability analysis is executed using `project.run()`, and the results are accessed from `project.design_point`.

In [69]:
project.settings.reliability_method = "form"

project.settings.relaxation_factor = 0.5
project.settings.maximum_iterations = 50
project.settings.variation_coefficient = 0.02

project.run()

def read_results(dp):

    beta = dp.reliability_index

    print(f"Beta = {beta}")

    pf = StandardNormal.get_q_from_u(beta)
    print(f"Probability of failure = {pf}")

    for alpha in dp.alphas:
        print(f"{alpha.variable.name}: alpha = {alpha.alpha}, x = {alpha.x}")

    if dp.is_converged:
        print(f"Converged (convergence = {dp.convergence} < {project.settings.variation_coefficient})")
    else:
        print(f"Not converged (convergence = {dp.convergence} > {project.settings.variation_coefficient})")

read_results(project.design_point)

Beta = 1.6270491218254477
Probability of failure = 0.05186333274069614
m: alpha = 0.8330742958548235, x = 0.4244242176477448
L: alpha = 0.048663450474983404, x = 49.802055439099206
c_creep: alpha = 0.0, x = 18.0
delta_H: alpha = -0.5510164118915687, x = 1.1743020359993066
Converged (convergence = 0.005402957046499092 < 0.02)


### Perform reliability calculations with Crude Monte Carlo

We now conduct the reliability analysis using the `Crude Monte Carlo` method. Here, we define the calculation settings: `minimum_samples`, `maximum_samples` and `variation_coefficient`.

In [70]:
project.settings.reliability_method = "crude_monte_carlo"

project.settings.minimum_samples = 1000
project.settings.maximum_samples = 50000
project.settings.variation_coefficient = 0.02

project.run()

read_results(project.design_point)

Beta = 1.644230562447786
Probability of failure = 0.05006429309218156
m: alpha = 0.8108514521738569, x = 0.4321182572644983
L: alpha = 0.047182916234664, x = 49.80605101775388
c_creep: alpha = 0.0, x = 18.0
delta_H: alpha = -0.583346976441243, x = 1.215495930680213
Converged (convergence = 0.01999932321915781 < 0.02)


### Perform reliability calculations with Importance Sampling
We now conduct the reliability analysis using the `Importance Sampling` method. Here, we define the calculation settings: `minimum_samples`, `maximum_samples` and `variation_coefficient`.

In [71]:
project.settings.reliability_method = "importance_sampling"

project.settings.minimum_samples = 1000
project.settings.maximum_samples = 50000
project.settings.variation_coefficient = 0.02

project.run()

read_results(project.design_point)

Beta = 1.6579312576083969
Probability of failure = 0.0486656711092089
m: alpha = 0.8198931538527077, x = 0.42309734419332007
L: alpha = 0.06107936685955877, x = 49.74683652122152
c_creep: alpha = 0.0, x = 18.0
delta_H: alpha = -0.5692490906532007, x = 1.2052883343114997
Not converged (convergence = 0.020015914943212608 > 0.02)
